# 코스피 데이터 스크래핑

In [7]:
import csv
import requests
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page='

filename = '시가총액1-200.csv'
f = open(filename, 'w', encoding = 'utf-8-sig', newline = '')
writer = csv.writer(f)


title = "N	종목명	현재가	전일비	등락률	액면가	시가총액	상장주식수	외국인비율	거래량	PER	ROE	토론실".split('\t')
print(type(title))
writer.writerow(title)

for page in range(1, 5):
    res = requests.get(url + str(page))
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')
    
    data_rows = soup.find('table', attrs = {'class' : 'type_2'}).find('tbody').find_all('tr')
    for row in data_rows:
        columns = row.find_all('td')
        if len(columns) <= 1:
            continue
        data = [column.get_text().strip() for column in columns]
        writer.writerow(data)

<class 'list'>


# 구글 무비

In [10]:
import requests
from bs4 import BeautifulSoup

url = 'https://play.google.com/store/movies/collection/cluster?clp=0g4XChUKD3RvcHNlbGxpbmdfcGFpZBAHGAQ%3D:S:ANO1ljJvXQM&gsr=ChrSDhcKFQoPdG9wc2VsbGluZ19wYWlkEAcYBA%3D%3D:S:ANO1ljK7jAA&hl=ko&gl=US'
res = requests.get(url)
res.raise_for_status()
soup = BeautifulSoup(res.text, 'lxml')

movies = soup.find_all('div', attrs = {'class' : 'ImZGtf mpg5gc'})
# print(len(movies))

for movie in movies:
    title = movie.find('div', attrs = {'class' : "WsMG1c nnK0zc"}).get_text()
    print(title)

50
The Hitman's Wife's Bodyguard
Wrath Of Man
The Forever Purge
Nobody
콰이어트 플레이스 2
고질라 VS. 콩
A Quiet Place
Werewolves Within
Spiral: From The Book of Saw
컨저링 3: 악마가 시켰다
The Hitman's Bodyguard
모탈 컴뱃
Demon Slayer - Kimetsu no Yaiba The Movie: Mugen Train (English Dubbed Version)
Midnight in the Switchgrass
Venom
Till Death
Out of Death
Demon Slayer - Kimetsu no Yaiba The Movie: Mugen Train (Original Japanese Version)
Cruella
Batman: The Long Halloween Part 1
A Quiet Place 2-Movie Collection
Great White
Escape Room
Zola
Space Jam
The House Next Door: Meet the Blacks 2
The Misfits
Spider-Man: Homecoming
The Marksman
라야와 마지막 드래곤
John Wick: Chapter 3 - Parabellum
Interstellar
Kung Fu Panda 3
Star Trek Into Darkness
Spider-Man: Into The Spider-Verse
Unholy, The
John Wick
Spider-Man: Far from Home
Fast & Furious Presents: Hobbs & Shaw
Sing
Live Die Repeat: Edge of Tomorrow
Minari
The Fate of the Furious
Closed for Storm
Lansky
Settlers
The Wolf of Wall Street
How It Ends
크루즈 패밀리: 뉴 에이지
Moana


In [24]:
from selenium import webdriver
browser = webdriver.Chrome()
browser.maximize_window()

url = 'https://play.google.com/store/movies/collection/cluster?clp=0g4XChUKD3RvcHNlbGxpbmdfcGFpZBAHGAQ%3D:S:ANO1ljJvXQM&gsr=ChrSDhcKFQoPdG9wc2VsbGluZ19wYWlkEAcYBA%3D%3D:S:ANO1ljK7jAA&hl=ko&gl=US'
browser.get(url)

# 지정한 위치로 스크롤 내리기
#browser.execute_script('window.scrollTo(0, 1080)')  # 1920 x 1080
#browser.execute_script('window.scrollTo(0, 2080)')

# 화면 가장 아래로 스크롤 내리기
# browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')

import time
interval = 2  # 2초에 한번씩 스크롤 내림

# 현재 문서 높이를 가져와서 저장
prev_height = browser.execute_script('return document.body.scrollHeight')

# 반복 수행
while True:
    # 스크롤을 가장 아래로 내림
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    
    # 페이지 로딩 대기
    time.sleep(interval)
    curr_height = browser.execute_script("return document.body.scrollHeight")
    if prev_height == curr_height:
        break
    
    prev_height = curr_height
    
print('스크롤 완료')

soup = BeautifulSoup(browser.page_source, 'lxml')

movies = soup.find_all('div', attrs = {'class' : 'Vpfmgd'})

for movie in movies:
    title = movie.find('div', attrs = {'class' : "WsMG1c nnK0zc"}).get_text()
    print(title)
    
    # 할인 전 가격
    original_price = movie.find('span', attrs={'class':'SUZt4c djCuy'})
    if original_price:
        original_price = original_price.get_text()
    else:
        #print(title, '   <할인되지 않은 영화 제외>')
        continue
        
    # 할인된 가격
    price = movie.find('span', attrs = {'class' : 'VfPpfd ZdBevf i5DZme'}).get_text()
    
    # 링크 정보
    link = movie.find('a', attrs = {'class' : 'JC71ub'})['href']
    # 올바른 링크 : 'https://play.google.com + link'
    
    print(f"제목 : {title}")
    print(f"할인 전 금액 : {original_price}")
    print(f"할인 후 금액 : {price}")
    print(f"링크 : ", "https://play.google.com" + link)
    print('-' * 100)
    
browser.quit()

스크롤 완료
The Hitman's Wife's Bodyguard
Wrath Of Man
제목 : Wrath Of Man
할인 전 금액 : US$19.99
할인 후 금액 : US$5.99
링크 :  https://play.google.com/store/movies/details/Wrath_Of_Man?id=V1Mo1-GV_No.P
----------------------------------------------------------------------------------------------------
The Forever Purge
Nobody
콰이어트 플레이스 2
고질라 VS. 콩
A Quiet Place
Werewolves Within
Spiral: From The Book of Saw
제목 : Spiral: From The Book of Saw
할인 전 금액 : US$19.99
할인 후 금액 : US$5.99
링크 :  https://play.google.com/store/movies/details/Spiral_From_The_Book_of_Saw?id=CRwU0_-TzT0.P
----------------------------------------------------------------------------------------------------
컨저링 3: 악마가 시켰다
The Hitman's Bodyguard
모탈 컴뱃
Demon Slayer - Kimetsu no Yaiba The Movie: Mugen Train (English Dubbed Version)
Midnight in the Switchgrass
Venom
Till Death
Out of Death
Demon Slayer - Kimetsu no Yaiba The Movie: Mugen Train (Original Japanese Version)
Cruella
Batman: The Long Halloween Part 1
A Quiet Place 2-Movie Collecti

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

browser = webdriver.Chrome()
browser.maximize_window()
url = 'https://play.google.com/store/movies/collection/cluster?clp=0g4XChUKD3RvcHNlbGxpbmdfcGFpZBAHGAQ%3D:S:ANO1ljJvXQM&gsr=ChrSDhcKFQoPdG9wc2VsbGluZ19wYWlkEAcYBA%3D%3D:S:ANO1ljK7jAA&hl=ko&gl=US'
browser.get(url)

################################## 스크롤 ##################################
interval = 2
prev_height = browser.execute_script('return document.body.scrollHeight')

while True:
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(interval)
    
    curr_height = browser.execute_script("return document.body.scrollHeight")
    if prev_height == curr_height:
        break
        
    prev_height = curr_height
    
############################################################################
################################# 영화정보 #################################
soup = BeautifulSoup(browser.page_source, 'lxml')

movies = soup.find_all('div', attrs = {'class' : 'Vpfmgd'})

for movie in movies:
    title = movie.find('div', attrs = {'class' : "WsMG1c nnK0zc"}).get_text()
    print(title)
    
    original_price = movie.find('span', attrs={'class':'SUZt4c djCuy'})
    if original_price:
        original_price = original_price.get_text()
    else:
        continue
        
    price = movie.find('span', attrs = {'class' : 'VfPpfd ZdBevf i5DZme'}).get_text()
    
    link = movie.find('a', attrs = {'class' : 'JC71ub'})['href']
    
    print(f"제목 : {title}")
    print(f"할인 전 금액 : {original_price}")
    print(f"할인 후 금액 : {price}")
    print(f"링크 : ", "https://play.google.com" + link)
    print('-' * 100)
    
browser.quit()